<a href="https://colab.research.google.com/github/KaggleStudy4th/4th_kaggle_study/blob/main/%EC%8B%A4%EC%8A%B5%EC%BD%94%EB%93%9C/3%EC%A3%BC%EC%B0%A8/Jiin/3D_MNIST_%EC%88%AB%EC%9E%90_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%matplotlib inline
import warnings  
warnings.filterwarnings('ignore')

import os
import json
import glob
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.graph_objs as go

from sklearn.svm import LinearSVC
from sklearn.preprocessing import MinMaxScaler

In [2]:
os.environ['KAGGLE_USERNAME']='jiinpp'
os.environ['KAGGLE_KEY']="b06d525a9f66198051ccb648b091778a"

In [3]:
!kaggle competitions download -c 2021-ml-p11

  0% 0.00/5.76k [00:00<?, ?B/s]
100% 5.76k/5.76k [00:00<00:00, 10.3MB/s]
 74% 33.0M/44.4M [00:01<00:00, 21.4MB/s]
100% 44.4M/44.4M [00:01<00:00, 28.8MB/s]


In [5]:
!unzip '*.zip'

Archive:  mnist3D_data.json.zip
  inflating: mnist3D_data.json       


In [6]:
with open("/content/mnist3D_data.json",'r') as j:
  data=json.load(j)

In [8]:
# 1720 번째 이미지에 대한 출력
image_number = 1720

digit = data["train"][image_number]["label"]
coordinate = np.array(data["train"][image_number]["points"])

layout = go.Layout(title = "Digit " + str(digit))
plot_data = go.Scatter3d(x = coordinate[:,0], y = coordinate[:,1], z = coordinate[:,2],
                    mode = 'markers', marker = dict(size = 3))

fig = go.Figure(data = [plot_data], layout = layout)
fig.show()

In [9]:
from torch.utils.data import Dataset

class MnistDataset(Dataset):
    def __init__(self, data, mode='train'):
        self.data = data[mode]
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        xyz = np.array(self.data[idx]['points'], dtype=np.float32)
        return {'points':xyz, 'label':self.data[idx]['label']}

In [10]:
# Voxel Grid 생성
# 목표 : point_cloud를 사용자가 지정한 voxel_size 크기의 voxel_grid로 분할한다

def VoxelGrid(points, voxel_size=[1, 1, 1]):

    # Voxel Grid를 구하기 위해 각 축 별 최대, 최소값 계산
    min_xyz  = np.min(points, axis=0) - 0.001
    max_xyz  = np.max(points, axis=0) + 0.001

    diff = max(max_xyz - min_xyz ) - ( max_xyz - min_xyz )
    min_xyz  = min_xyz  - diff / 2
    max_xyz  = max_xyz  + diff / 2

    # voxel_grid
    voxel_grid = []

    # point cloud를 각 축 (x,y,z) 에 대한 voxel grid 로 분할
    for i in range(3):
        # Voxel_size는 정수만 가능
        if type(voxel_size[i]) is not int:
            raise TypeError("voxel_size[{}] must be int".format(i))
        segment = np.linspace( min_xyz[i], max_xyz[i], num=(voxel_size[i] + 1))
        voxel_grid.append(segment)

    return voxel_grid # 생성한 voxel_grid 를 반환

In [11]:
# Voxelization (복셀화) 및 Voxel_Vector 추출
# 목표: 생성한 voxel_grid에 맞추어 point cloud를 각 grid에 맞춰 Voxelization하고, 이를 통해 voxel_vector 

def VoxelVector(points, voxel_grid, voxel_size=[1, 1, 1]):

    n_voxels = voxel_size[0] * voxel_size[1] * voxel_size[2]
    n_x = voxel_size[0]
    n_y = voxel_size[1]
    n_z = voxel_size[2]

    voxelization = np.zeros((len(points), 4), dtype=int)

    # Voxelization
    voxelization[:,0] = np.searchsorted(voxel_grid[0], points[:,0]) - 1
    voxelization[:,1] = np.searchsorted(voxel_grid[1], points[:,1]) - 1
    voxelization[:,2] = np.searchsorted(voxel_grid[2], points[:,2]) - 1

    voxelization[:,3] = ((voxelization[:,1] * n_x) + voxelization[:,0]) + (voxelization[:,2] * (n_x * n_y)) 

    # Voxelization으로부터 voxel_vector 추출
    vector = np.zeros(n_voxels)
    count = np.bincount(voxelization[:,3])
    vector[:len(count)] = count

    vector = vector.reshape(n_z, n_y, n_x)

    return vector # 추출한 Vector 반환

In [12]:
# Voxel_Feature 추출
# 목표: 추출한 voxel_vector로부터 classification에 feature로 활용할 수 있는 voxel_feature를 추출한다

def VoxelFeature(dataset):
    
    voxel_vectors = []
    labels = []

    for i, pc in enumerate(tqdm(dataset)):

        # point cloud를 np.array(dtype=np.float32)로 변경
        pointcloud = np.array(pc['points'], dtype = np.float32)   

        voxel_grid = VoxelGrid(pointcloud, voxel_size=[8, 8, 8])
        
        vector = VoxelVector(pointcloud, voxel_grid, voxel_size=[8, 8, 8])

        voxel_vectors.append(vector.reshape(-1) / np.max(vector))
        
        labels.append(pc['label'])

    return np.array(voxel_vectors), np.array(labels)

## Model/Evaluation

In [13]:
# [1] 데이터 불러온 후
mnist_train = MnistDataset(data, mode = "train")
mnist_test = MnistDataset(data, mode = "test")

# [2] 불러온 Raw 데이터를 가공합니다.
x_train, y_train = VoxelFeature(mnist_train)
x_test, _ = VoxelFeature(mnist_test)

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [14]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

(5000, 512)
(5000,)
(1000, 512)


In [18]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

scaler=MinMaxScaler()
x_train_scale=scaler.fit_transform(x_train)
x_test_scale=scaler.transform(x_test)

svc=SVC(random_state=1)
svc.fit(x_train_scale,y_train)
y_svc_predict=svc.predict(x_test_scale)

sample=pd.read_csv('/content/sample_submit.csv')
sample['Label']=y_svc_predict
sample.to_csv('svc.csv',index=False)